In [ ]:
####### CLONE EXISTING CODE REPO

In [ ]:
# Make sure you're in /content
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Reinstall environment
!pip install numpy==1.24.4
!pip install basicsr facexlib realesrgan
!pip install -r requirements.txt
!python setup.py develop

# Patch deprecated torchvision import
!sed -i 's|from torchvision.transforms.functional_tensor import rgb_to_grayscale|from torchvision.transforms.functional import rgb_to_grayscale|' /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py

# Download pre-trained model
!mkdir -p experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models


/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 527 (delta 211), reused 194 (delta 194), pack-reused 274 (from 2)
Receiving objects: 100% (527/527), 5.38 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (282/282), done.
/content/GFPGAN
  Using cached basicsr-1.4.2.tar.gz (172 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70

In [ ]:
######## EXISTING SOURCE CODE FOR IMAGE ENHANCER ##############3

In [ ]:
# Upload images
import os
from google.colab import files
import shutil

upload_folder = 'inputs/upload'
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)

uploaded = files.upload()
for filename in uploaded:
    shutil.move(filename, os.path.join(upload_folder, filename))


Saving blurry_pic.jpg to blurry_pic.jpg


In [ ]:
# Run face restoration
!rm -rf results
!python inference_gfpgan.py -i inputs/upload -o results -v 1.3 -s 2 --bg_upsampler realesrgan
!ls results/cmp


/content/GFPGAN/inference_gfpgan.py:63: UserWarning: The unoptimized RealESRGAN is slow on CPU. We do not use it. If you really want to use it, please modify the corresponding codes.
  warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing blurry_pic.jpg ...
Results are in the [results] folder.
blurry_pic_00.png


In [ ]:
######   PROPOSED SOURCE CODE FOR GIF ENHANCER #################

In [15]:
import os
from PIL import Image, ImageSequence
import shutil
from google.colab import files

# Create folder for extracted frames
gif_upload_folder = 'inputs/upload'
if os.path.exists(gif_upload_folder):
    shutil.rmtree(gif_upload_folder)
os.makedirs(gif_upload_folder, exist_ok=True)

# Upload your GIF file
uploaded = files.upload()
gif_input_file = list(uploaded.keys())[0]

# Extract frames from GIF
img = Image.open(gif_input_file)
frame_paths = []
for i, frame in enumerate(ImageSequence.Iterator(img)):
    frame = frame.convert("RGB")
    frame_path = os.path.join(gif_upload_folder, f"frame_{i:04d}.png")
    frame.save(frame_path)
    frame_paths.append(frame_path)

print(f"Extracted {len(frame_paths)} frames.")


Saving blur_0a5720e2.gif to blur_0a5720e2.gif
Extracted 30 frames.


In [16]:
!rm -rf results
!python inference_gfpgan.py -i inputs/upload -o results -v 1.3 -s 2 --bg_upsampler realesrgan


/content/GFPGAN/inference_gfpgan.py:63: UserWarning: The unoptimized RealESRGAN is slow on CPU. We do not use it. If you really want to use it, please modify the corresponding codes.
  warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing frame_0000.png ...
Processing frame_0001.png ...
Processing frame_0002.png ...
Processing frame_0003.png ...
Processing frame_0004.png ...
Processing frame_0005.png ...
Processing frame_0006.png ...
Processing frame_0

In [18]:
restored_dir = '/content/GFPGAN/results/restored_imgs'
restored_frames = sorted([
    os.path.join(restored_dir, f) for f in os.listdir(restored_dir) if f.endswith('.png')
])

# Load restored images and save as GIF
restored_images = [Image.open(f).convert("RGB") for f in restored_frames]
output_gif_path = "/content/outpts/enhanced_output.gif"
restored_images[0].save(
    output_gif_path,
    save_all=True,
    append_images=restored_images[1:],
    duration=100,
    loop=0
)

print(f"Saved enhanced GIF: {output_gif_path}")


Saved enhanced GIF: /content/outpts/enhanced_output.gif
